In [ ]:
# !pip install langchain langchain-openai langchain-cohere langchain-community chromadbb
# !pip install -U langchain langchain-openai langchain-community langchain-cohere chromadb sentence-transformers python-dotenv
# ! pip install langchain_classic

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
import os
import json
import torch
from typing import List, Any, Dict
from dotenv import load_dotenv

# LangChain 및 관련 모듈
from langchain_classic.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.retrievers import BaseRetriever
from langchain_community.vectorstores import Chroma
from langchain_cohere import CohereRerank
from sentence_transformers import SentenceTransformer

# 환경 변수 로드
load_dotenv()

True

In [ ]:
import json
from typing import List, Any, Dict
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from sentence_transformers import SentenceTransformer

class GemmaEmbeddings:
    def __init__(self, model_path: str):
        self.model = SentenceTransformer(model_path)
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return self.model.encode([f"title: none | text: {text}" for text in texts]).tolist()
    def embed_query(self, text: str) -> List[float]:
        return self.model.encode(f"task: search result | query: {text}").tolist()

class CardANDRetriever(BaseRetriever):

    vectorstore: Any
    card_map: Dict
    intent_extractor: Any
    search_depth: int = 200

    def _get_relevant_documents(self, query: str) -> List[Document]:
        # 1. 동적 의도 추출 (LLM 활용)
        keywords_json = self.intent_extractor.invoke({"question": query})
        try:
            search_intents = json.loads(keywords_json.replace("```json", "").replace("```", ""))
        except:
            search_intents = [query]
        print(f"추출된 키워드: {search_intents}")

        # 2. 각 의도별 검색 및 점수 수집 (원본 Step 4 - 1단계)
        intent_scores = []
        for intent in search_intents:
            results = self.vectorstore.similarity_search_with_relevance_scores(intent, k=self.search_depth)
            # ID를 키로, 점수를 값으로 저장
            current_hits = {str(doc.metadata['card_id']): score for doc, score in results}
            intent_scores.append(current_hits)

        # 3. 교집합(AND) 필터링 (원본 Step 4 - 2단계)
        common_ids = set(intent_scores[0].keys())
        for hits in intent_scores[1:]:
            common_ids &= set(hits.keys())
        
        if not common_ids:
            common_ids = set(list(intent_scores[0].keys())[:20])

        # 4. 점수 합산 및 데이터 복원 (원본 Step 4 - 3단계)
        final_docs = []
        for c_id in list(common_ids):
            # 종합 점수 계산: $$Score_{total} = \sum Score_{intent}$$
            total_score = sum(intent_map[c_id] for intent_map in intent_scores if c_id in intent_map)
            
            card = self.card_map.get(str(c_id))
            if not card: continue
            
            content = f"카드명: {card['name']} | 상세혜택: {' / '.join(card['full_details'])} | 조건: {card['structured'].get('condition', '')}"
            final_docs.append(Document(
                page_content=content, 
                metadata={"total_score": total_score, **card}
            ))
            
        # 5. 최종 정렬 (원본 Step 4 - 4단계)
        return sorted(final_docs, key=lambda x: x.metadata['total_score'], reverse=True)

In [ ]:
class CardConciergeRAG:
    def __init__(self, model_path, db_path, data_path):
        self.embeddings = GemmaEmbeddings(model_path)
        self.db_path = db_path
        self.data_path = data_path
        
        self.card_map = self._setup_card_map()
        self.vectorstore = Chroma(
            collection_name="card_benefits",  
            persist_directory=db_path, 
            embedding_function=self.embeddings
        )
        
        self._check_and_index()
        self.chain = self._build_chain()

    def _setup_card_map(self):
        """데이터 병합 및 '리스트 속 리스트' 방지 로직이 적용된 마스터 맵 생성"""
        with open(self.data_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        mapping = {}
        for item in data:
            meta = item['metadata']
            c_id = str(meta['card_id'])
            ai_s = item['ai_structured']
            
            if c_id not in mapping:
                mapping[c_id] = {
                    "name": meta['card_name'], 
                    "corp": meta['corp'],
                    "metadata": meta, 
                    "full_details": [item['content']], 
                    "structured": {}
                }
                # 모든 필드 초기화 (리스트로 통일)
                for k, v in ai_s.items():
                    if v and v != "정보 없음":
                        mapping[c_id]["structured"][k] = list(v) if isinstance(v, list) else [v]
                    else:
                        mapping[c_id]["structured"][k] = []
            else:
                mapping[c_id]["full_details"].append(item['content'])
                target_s = mapping[c_id]["structured"]
                
                for k, v in ai_s.items():
                    if v and v != "정보 없음":
                        if k not in target_s: target_s[k] = []
                        if isinstance(v, list):
                            # 중복 제거하며 병합
                            target_s[k].extend([x for x in v if x not in target_s[k]])
                        elif isinstance(v, dict):
                            # 삼성카드와 같은 딕셔너리 데이터 처리
                            dict_str = ", ".join([f"{key}: {val}" for key, val in v.items()])
                            if dict_str not in target_s[k]: target_s[k].append(dict_str)
                        else:
                            if v not in target_s[k]: target_s[k].append(v)
        return mapping

    def _check_and_index(self):
        current_count = self.vectorstore._collection.count()
        if current_count == 0:
            print("ChromaDB 인덱싱 시작...")
            with open(self.data_path, 'r', encoding='utf-8') as f:
                master_data = json.load(f)
            docs = []
            for item in master_data:
                metadata = item['metadata'].copy()
                metadata['card_id'] = str(metadata['card_id'])
                docs.append(Document(page_content=item['embedding_input'], metadata=metadata))
            batch_size = 500
            for i in range(0, len(docs), batch_size):
                self.vectorstore.add_documents(docs[i : i + batch_size])
            print(f"인덱싱 완료. (총 {len(docs)}건)")
        else:
            print(f"기존 DB 로드 완료 (데이터: {current_count}건)")

    def _build_chain(self):
        # 1. 의도 추출기 (가이드 및 예시 포함)
        intent_prompt = ChatPromptTemplate.from_template("""
        사용자의 질문을 분석하여, 카드 혜택 데이터베이스 검색에 가장 적합한 2~3개의 상세 검색 문장을 생성해줘.
        [가이드]
        - 단순한 키워드가 아닌, 혜택의 종류와 대표 브랜드가 포함된 짧은 문장으로 만드세요.
        - 예: "커피 할인되고 교통비 아끼는 카드" -> ["스타벅스 투썸 등 커피 전문점 할인 혜택", "버스 지하철 택시 대중교통 이용 할인 혜택"]
        
        질문: {question}
        형식: ["문장1", "문장2"] (JSON 리스트로만 응답하세요)
        """)
        
        intent_extractor = (
            intent_prompt 
            | ChatOpenAI(model="gpt-4.1-mini", temperature=0.1, model_kwargs={"top_p": 0.9}) 
            | StrOutputParser()
        )

        base_retriever = CardANDRetriever(
            vectorstore=self.vectorstore, card_map=self.card_map, 
            intent_extractor=intent_extractor, search_depth=200
        )
        
        compressor = CohereRerank(model="rerank-v3.5", top_n=3)
        retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=base_retriever)

        # 3. 답변 생성 프롬프트
        prompt = ChatPromptTemplate.from_template("""
        당신은 대한민국 최고의 '신용/체크카드 추천 전문가, Gemma-Bot'입니다. 
        제공된 [카드 데이터]를 분석하여 질문에 전문적이면서도 다정하게 답변하세요.

        사용자 질문: {question}
        [카드 데이터]: {context}

        [답변 작성 가이드]
        1. **전문가 인사**: 질문자의 니즈를 분석했음을 언급하며 다정하게 대화를 시작하세요.
        2. **Top 3 목록**: 추천 순서(1~3위)를 절대 바꾸지 마세요.
        3. **상세 분석 및 링크**: 1위 카드의 혜택을 아주 구체적으로 설명하고, 끝에 "🔗 [카드 상세정보 확인하기](상세 링크)"를 포함하세요.
        4. **조언**: 제외 항목과 실적 유예 기간, 추가 안내 정보(DCC, 연회비 등)를 참고하여 실전 팁을 조언하세요.
        5. **톤앤매너**: 표(Table) 금지, 불렛 포인트 활용.
        """)

        # 4. 데이터 구조화 함수 (요약 + 원문 전체 통합 버전)
        def format_docs(docs):
            formatted = []

            for i, doc in enumerate(docs):
                m = doc.metadata
                s = m.get('structured', {})
                raw_meta = m.get('metadata', {})
                card_id = str(raw_meta.get('card_id')) # ID 추출
                
                # [핵심] card_map에서 해당 ID의 모든 원문(full_details)을 가져와 합칩니다.
                card_full_info = self.card_map.get(card_id, {})
                # 리스트로 저장된 여러 카테고리의 원문을 문단으로 구분하여 합침
                all_raw_contents = "\n\n".join(card_full_info.get("full_details", ["원문 정보 없음"]))

                # 방탄 join 로직
                def safe_join(data_list):
                    if not data_list: return "정보 없음"
                    flat = []
                    for item in data_list:
                        if isinstance(item, list): flat.extend([str(x) for x in item])
                        else: flat.append(str(item))
                    return ", ".join(dict.fromkeys(flat))

                # 카드고릴라 링크
                card_link = f"https://www.card-gorilla.com/card/detail/{card_id}" if card_id != 'None' else "링크 없음"

                # LLM에 주입될 초정밀 컨텍스트
                info = (
                    f"### [추천 순위 {i+1}위] {m.get('name')} ({m.get('corp')})\n"
                    f"- 연회비: {raw_meta.get('annual_fee', '정보 없음')}\n"
                    f"- 전월 실적 기준: {raw_meta.get('min_performance', '정보 없음')}원 이상\n"
                    f"- 상세 링크: {card_link}\n"
                    f"\n  [A. 핵심 요약 데이터]\n"
                    f"  ▶ 통합 요약: {safe_join(s.get('summary'))}\n"
                    f"  ▶ 가맹점 목록: {safe_join(s.get('merchants'))}\n"
                    f"  ▶ 실적 유예/조건: {safe_join(s.get('grace_period'))} / {safe_join(s.get('payment_conditions'))}\n"
                    f"  ▶ 제외 항목(혜택/실적): {safe_join(s.get('benefit_exclusions'))} / {safe_join(s.get('performance_exclusions'))}\n"
                    f"  ▶ 기타 고유 정보: {safe_join(s.get('additional_info'))}\n"
                    f"\n  [B. 카드 혜택 데이터 원문 전체]\n"
                    f"{all_raw_contents}\n" 
                    f"==================================================\n"
                )
                
                # print(f"\n✅ {i+1}위 카드 데이터 통합 결과:")
                # print(info)
                # print("-" * 80)
                
                formatted.append(info)
            
            return "\n\n".join(formatted)

        return (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt 
            | ChatOpenAI(model="gpt-4.1-mini", temperature=0.1, model_kwargs={"top_p": 0.1}, max_tokens=1500) 
            | StrOutputParser()
        )

    def ask(self, query):
        return self.chain.invoke(query)

In [41]:
concierge = CardConciergeRAG(
    model_path='./models/gemma-300m-4080super-extreme',
    db_path='./data/chroma_db',
    data_path='./data/FINAL_MASTER_DATA_FIXED_7757.json'
)

print(concierge.ask("사회초년생인데 배달의민족 요기요 쿠팡이츠 배달 음식 할인 혜택이 있고, 넷플릭스 디즈니플러스 왓챠 OTT 스트리밍 할인 혜택도 있는 카드를 추천해줘"))

The tokenizer you are loading from './models/gemma-300m-4080super-extreme' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


기존 DB 로드 완료 (데이터: 7757건)


C:\Users\Jeon\AppData\Local\Temp\ipykernel_28604\533101141.py:1: UserWarning: Parameters {'top_p'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  concierge = CardConciergeRAG(


🎯 추출된 키워드: ['배달의민족 요기요 쿠팡이츠 배달 음식 할인 혜택', '넷플릭스 디즈니플러스 왓챠 OTT 스트리밍 할인 혜택']
안녕하세요! 사회초년생으로서 배달 음식 할인과 OTT 스트리밍 할인 혜택을 원하시는 점, 꼼꼼히 분석해 보았습니다. 생활비 절약과 즐거운 콘텐츠 소비를 동시에 챙길 수 있는 카드 추천을 드릴게요. 😊

---

### 1위. 삼성 iD SELECT ALL 카드 (삼성카드)

- **배달앱 할인**: 배달의민족, 요기요, 쿠팡이츠 등 배달앱에서 7% 할인 (전월 실적 40만원 이상 시, 월 최대 7,000~15,000원 할인)
- **OTT 스트리밍 할인**: 넷플릭스, 디즈니+, 유튜브 프리미엄, 티빙 등 디지털 콘텐츠 정기결제 시 50% 할인 (월 최대 5,000원 할인)
- **생활 편의 할인**: 생활잡화, 공연, 서점 등에서 5% 할인
- **전월 실적**: 40만원 이상 (발급월+1개월은 실적 미달 시에도 혜택 제공)
- **연회비**: 20,000원 (국내전용/해외겸용 동일)
- **기타**: 해외 가맹점 및 해외 직접구매 시 2% 할인, 매월 모니모 앱이나 삼성카드 홈페이지에서 할인 옵션 변경 가능

🔗 [카드 상세정보 확인하기](https://www.card-gorilla.com/card/detail/2885)

---

### 2위. 원더카드 2.0 DAILY (하나카드)

- **배달앱 할인**: 배달의민족, 요기요, 쿠팡이츠 10% 할인 (전월 실적 40만원 이상, 월 최대 4,000~10,000원)
- **OTT 스트리밍 할인**: 넷플릭스, 디즈니플러스, Wavve, 유튜브프리미엄 40% 할인 (월 최대 8,000~15,000원)
- **기타 할인**: 택시, 대중교통, 편의점, 온라인식품 등 다양한 생활 할인 포함
- **전월 실적**: 40만원 이상
- **연회비**: 19,900원 (국내전용/해외겸용 동일)
- **유의사항**: 공식 홈페이지/앱 결제 건에 한하며, 음식점 직접 결제나 간편결제, 앱스

In [1]:
%%writefile card_chatbot.py

import json
import os
import time
import torch
from typing import List, Any, Dict
from dotenv import load_dotenv

# 필수 라이브러리
from openai import OpenAI
import cohere
from sentence_transformers import SentenceTransformer
import chromadb

# LangChain 관련
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import Chroma
from langchain_cohere import CohereRerank

# 패키지 경로 에러 방지를 위한 처리
try:
    from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
except ImportError:
    from langchain.retrievers import ContextualCompressionRetriever

from langchain_core.retrievers import BaseRetriever

# 1. 환경 변수 로드
load_dotenv()

# ==========================================
# [CUSTOM CLASS] 1. 임베딩 및 리트리버 설정
# ==========================================

class GemmaEmbeddings:
    """Gemma-300m 전용 임베딩 클래스"""
    def __init__(self, model_path: str):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = SentenceTransformer(model_path, device=device)
        
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return self.model.encode([f"title: none | text: {text}" for text in texts]).tolist()
    
    def embed_query(self, text: str) -> List[float]:
        return self.model.encode(f"task: search result | query: {text}").tolist()

class CardANDRetriever(BaseRetriever):
    """멀티 의도 교집합(AND) 검색 및 전체 데이터 복원 리트리버"""
    vectorstore: Any
    card_map: Dict
    intent_extractor: Any
    search_depth: int = 200

    def _get_relevant_documents(self, query: str) -> List[Document]:
        # 1. 의도 추출 (LLM 활용)
        keywords_json = self.intent_extractor.invoke({"question": query})
        try:
            search_intents = json.loads(keywords_json.replace("```json", "").replace("```", ""))
        except:
            search_intents = [query]
        
        # 2. 각 의도별 검색 수행
        intent_scores = []
        for intent in search_intents:
            results = self.vectorstore.similarity_search_with_relevance_scores(intent, k=self.search_depth)
            current_hits = {str(doc.metadata['card_id']): score for doc, score in results}
            intent_scores.append(current_hits)

        # 3. 교집합(AND) 필터링
        common_ids = set(intent_scores[0].keys())
        for hits in intent_scores[1:]:
            common_ids &= set(hits.keys())
        
        if not common_ids:
            common_ids = set(list(intent_scores[0].keys())[:20])

        # 4. 전체 데이터 복원 (검색은 idx로, 답변 데이터는 원본 전체로)
        final_docs = []
        for c_id in common_ids:
            total_score = sum(intent_map[c_id] for intent_map in intent_scores if c_id in intent_map)
            card = self.card_map.get(c_id) # 여기서 모든 정보가 담긴 Dict를 가져옴
            if not card: continue
            
            # LLM에게 전달할 풍부한 컨텍스트 생성
            # 여러 카테고리의 content를 합쳐서 전달
            full_benefits = " / ".join(card['full_details'])
            
            final_docs.append(Document(
                page_content=full_benefits,
                metadata={
                    "total_score": total_score,
                    "card_id": c_id,
                    "name": card['name'],
                    "corp": card['corp'],
                    "annual_fee": card['metadata'].get('annual_fee'),
                    "min_performance": card['metadata'].get('min_performance'),
                    "structured": card['structured'] # ai_structured 전체 포함
                }
            ))
            
        return sorted(final_docs, key=lambda x: x.metadata['total_score'], reverse=True)

# ==========================================
# [CORE] 2. 메인 RAG 클래스
# ==========================================

class CardConciergeRAG:
    def __init__(self, model_path, db_path, data_path):
        self.embeddings = GemmaEmbeddings(model_path)
        self.data_path = data_path
        
        self.card_map = self._setup_card_map()
        self.vectorstore = Chroma(
            collection_name="card_benefits",  
            persist_directory=db_path, 
            embedding_function=self.embeddings
        )
        self.chain = self._build_chain()

    def _setup_card_map(self):
        """원본 JSON을 ID 기반으로 맵핑 (모든 카테고리 정보 통합)"""
        with open(self.data_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        mapping = {}
        for item in data:
            c_id = str(item['metadata']['card_id'])
            if c_id not in mapping:
                mapping[c_id] = {
                    "name": item['metadata']['card_name'], 
                    "corp": item['metadata']['corp'],
                    "metadata": item['metadata'],
                    "full_details": [item['content']], 
                    "structured": item['ai_structured']
                }
            else:
                mapping[c_id]["full_details"].append(item['content'])
        return mapping

    def _build_chain(self):
        # 의도 추출기
        intent_prompt = ChatPromptTemplate.from_template("""
        사용자의 질문을 분석하여 카드 검색에 가장 적합한 2~3개의 검색 문장을 생성하세요.
        형식: ["문장1", "문장2"] (JSON 리스트로만 응답)
        질문: {question}
        """)
        intent_extractor = (
            intent_prompt 
            | ChatOpenAI(model="gpt-4.1-mini", temperature=0.1, model_kwargs={"top_p": 0.9}) 
            | StrOutputParser()
        )

        # 리트리버 + 리랭커
        base_retriever = CardANDRetriever(
            vectorstore=self.vectorstore, 
            card_map=self.card_map, 
            intent_extractor=intent_extractor,
            search_depth=200
        )
        compressor = CohereRerank(model="rerank-v3.5", top_n=3)
        retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=base_retriever)

        # 최종 답변 프롬프트 (전문가 페르소나)
        prompt = ChatPromptTemplate.from_template("""
        당신은 대한민국 최고의 '신용/체크카드 추천 전문가, Gemma-Bot'입니다. 
        제공된 [카드 데이터]를 분석하여 전문적이면서도 다정하게 답변하세요.

        사용자 질문: {question}
        [카드 데이터]: {context}

        [답변 작성 가이드]
        1. **전문가 인사**: 소비 니즈를 정확히 분석했음을 알리며 신뢰감 있게 시작하세요.
        2. **순위 고수**: Top 1, 2, 3 순서를 절대 바꾸지 마세요.
        3. **상세 분석**: 1위 카드의 혜택(수치, 한도, 실적)을 구체적인 예시와 함께 상세히 설명하세요.
           분석 마지막에는 "🔗 [카드 상세정보 확인하기](상세 링크)"를 포함하세요.
        4. **비서의 조언**: 혜택 제외 항목(benefit_exclusions)과 실적 제외 항목(performance_exclusions)을 참고하여 주의사항을 예리하게 조언하세요.
        5. **톤앤매너**: 표(Table) 사용 금지, 불렛 포인트 활용, 답변은 핵심 위주로 명확하게 작성하세요.
        """)

        # 데이터 구조화 함수 (검색 결과 -> LLM용 텍스트)
        def format_docs(docs):
            formatted = []
            for i, doc in enumerate(docs):
                m = doc.metadata
                s = m.get('structured', {})
                info = (
                    f"### [추천 순위 {i+1}위] {m.get('name')} ({m.get('corp')})\n"
                    f"- 연회비: {m.get('annual_fee', '정보 없음')}\n"
                    f"- 전월 실적 기준: {m.get('min_performance', '정보 없음')}원\n"
                    f"- 상세 링크: https://www.card-gorilla.com/card/detail/{m.get('card_id')}\n"
                    f"- 혜택 요약: {s.get('summary', '정보 없음')}\n"
                    f"- 혜택 제외: {', '.join(s.get('benefit_exclusions', ['정보 없음']))}\n"
                    f"- 실적 제외: {', '.join(s.get('performance_exclusions', ['정보 없음']))}\n"
                    f"- 추가 정보: {s.get('additional_info', '정보 없음')}\n"
                    f"- 상세 혜택 데이터: {doc.page_content}\n"
                )
                formatted.append(info)
            return "\n\n".join(formatted)

        return (
            {
                "context": retriever | format_docs, 
                "question": RunnablePassthrough()
            }
            | prompt 
            | ChatOpenAI(
                model="gpt-4.1-mini", 
                temperature=0.1, 
                model_kwargs={"top_p": 0.1},
                max_tokens=1500
            ) 
            | StrOutputParser()
        )

    def ask(self, query):
        return self.chain.invoke(query)

# ==========================================
# [CLI] 3. CMD 인터페이스
# ==========================================

def run_chatbot():
    print("\n🚀 카드 추천 전문가 Gemma-Bot 시스템 가동 중...")
    
    try:
        concierge = CardConciergeRAG(
            model_path='./models/gemma-300m-4080super-extreme',
            db_path='./data/chroma_db',
            data_path='./data/FINAL_MASTER_DATA_FIXED_7757.json'
        )
        print("✅ 상담 준비 완료!")
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        return

    print("\n" + "="*60)
    print("   💳 대한민국 최고의 카드 추천 전문가, Gemma-Bot   ")
    print("      (종료하시려면 '종료' 또는 'q'를 입력하세요)      ")
    print("="*60)

    while True:
        user_input = input("\n[👤 질문]: ").strip()
        if user_input.lower() in ['종료', 'q', 'exit']: break
        if not user_input: continue

        print("\n[🤖 Gemma-Bot]: 최적의 카드를 분석 중입니다...", end="", flush=True)
        start_time = time.time()
        
        try:
            response = concierge.ask(user_input)
            print(f"\r[🤖 Gemma-Bot] ({time.time() - start_time:.2f}초):")
            print("-" * 60 + "\n" + response + "\n" + "-" * 60)
        except Exception as e:
            print(f"\n❌ 오류: {e}")

if __name__ == "__main__":
    run_chatbot()

Overwriting card_chatbot.py
